In [641]:
import argparse
#from tesseract import image_to_string
import os
import pytesseract
import collections
import pandas as pd
from PIL import Image
import math
import numpy as np
from tqdm import tqdm_notebook as tqdm
from pandas import DataFrame
import imgaug.augmenters as iaa
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [407]:
folder_tsv = "/home/andreizoltan/vision/github/annotations/training/"

number_of_needed_labels = 0
text_on_labels = list()
labels_dict = dict()
counter = collections.Counter()
labels_with_data = set()
def main():
    global k2, success, number_of_needed_labels, text_on_labels, labels_dict
    for folder in tqdm(os.listdir(folder_tsv)):
        for file in os.listdir(folder_tsv + folder):
            df = DataFrame.from_csv(folder_tsv + folder + "/" + file, sep='\t')
            for index, row in df.iterrows():
                if isinstance(row["data"], float) == False and row["data"] != "":
                    if not index in labels_dict:
                        labels_dict[index] = [row["data"]]
                    else:
                        list_of_label = labels_dict[index]
                        list_of_label.append(row["data"])
                        labels_dict[index] = list_of_label

In [408]:
main()

/home/andreizoltan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  if sys.path[0] == '':


In [409]:
labels_with_data = set()
most_common = dict()
for key in labels_dict.keys():
    list_of_label = labels_dict[key]
    counter = collections.Counter()
    if isinstance(key, float) == True:
        if math.isnan(key) == True:
            continue
        else:
            labels_with_data.add(str(key))
    elif isinstance(key, pandas._libs.tslibs.timestamps.Timestamp) == True:
        continue
    else:
        labels_with_data.add(key)

    for text in list_of_label:
        counter[text] += 1
    most_common[str(key)] = str(counter.most_common()[0][0])
print(labels_with_data)
print(most_common)

{'8.2.1', '3.4', '6.10.2', '8.11', '8.2.6', '3.25', '5.32', '8.2.5', '8.4.1', '6.9.2', '8.1.4', '8.1.1', '8', '5.31', '8.5.4', '3.13', '6.13', '7.5', '8.2.2', '3.11', '6.9.1', '5.23.1', '1.13', '6.10.1', '6.11', '5.24.1', '3.24'}
{'8.2.1': '400 м', '5.23.1': 'Заречье', '3.24': '20', '1.13': '6 %', '7.5': '200 м', '6.10.1': 'усадьба “Назарьево” “Nazarevo” estate', '8.1.1': '250 м', '3.13': '4,5 м', '6.9.2': 'М9 Псков М3 Калуга МКАД', '6.10.2': '53', '8.2.2': '70 м', '8.1.4': '50 м', '3.4': '3,5 т', '3.25': '80', '5.24.1': 'Ново-Ивановское', '6.13': '37 мосавтодор', '8.4.1': '12 т без пропуска', '8.5.4': '6.00 - 22.00', '8': 'Пропан', '6.9.1': 'МКАД Сколковское ш. 0.6 Ленинский пр-т 8.3', '8.2.5': '600 м', '5.31': '20', '5.32': '40', '6.11': 'р. Сетунь', '8.11': '12 т', '3.11': '5 т', '8.2.6': '50 м'}


In [619]:
def get_data(label):
    label = str(label)
    if label in labels_with_data:
        return most_common[str(label)]
    else:
        return ""

In [653]:
def add_data(file):
    df = pd.read_csv(file, sep='\t', dtype=str)
    df['data'] = df['class'].apply(get_data)
    new_file = "/home/andreizoltan/vision/github/icevision/icevision/annot_new.tsv"
    df.to_csv(new_file, sep='\t', index=False)

In [654]:
def check(df):
    for index, row in df.iterrows():
        if row['data'] != 0:
            print(type(row['data']))

In [655]:
file = "/home/andreizoltan/vision/github/icevision/icevision/annot.tsv"
add_data(file)